# ELECTRA training

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None

### Choosing starting point: BM25 or T5 top-k passages

In [7]:
# Reimporting the whole dataset with results of T5 predictions to choose positive and negative examples
import ast
df_T5_100 = pd.read_csv('df_T5_top100.csv')
df_T5_100['T5Top100Passages'] = df_T5_100['T5Top100Passages'].apply(lambda x: ast.literal_eval(x))
df_T5_100['T5posPassages'] = df_T5_100['T5posPassages'].apply(lambda x: ast.literal_eval(x))
df_T5_100['T5negPassages'] = df_T5_100['T5negPassages'].apply(lambda x: ast.literal_eval(x))
df_T5_100['bm25TopPassages'] = df_T5_100['bm25TopPassages'].apply(lambda x: ast.literal_eval(x))
df_T5_100['passages'] = df_T5_100['passages'].apply(lambda x: ast.literal_eval(x))
#df_T5_100['answerPage'] = df_T5_100['answerPage'].apply(lambda x: ast.literal_eval(x))

In [8]:
# Reimporting the whole dataset with results of BM25 predictions to choose positive and negative examples
import ast
df_bm25_full = pd.read_csv('df_bm25_full.csv')
df_bm25_full['bm25TopPassages'] = df_bm25_full['bm25TopPassages'].apply(lambda x: ast.literal_eval(x))
df_bm25_full['bm25posPassages'] = df_bm25_full['bm25posPassages'].apply(lambda x: ast.literal_eval(x))
df_bm25_full['bm25negPassages'] = df_bm25_full['bm25negPassages'].apply(lambda x: ast.literal_eval(x))
df_bm25_full['passages'] = df_bm25_full['passages'].apply(lambda x: ast.literal_eval(x))
#df_bm25_full['answerPage'] = df_bm25_full['answerPage'].apply(lambda x: ast.literal_eval(x))

In [9]:
# Re-computing anwserPages since ast causing error during reimport
def removeFakePgNbrs(pageNbr):
    pageNbr = pageNbr.lower().replace("l0", "").replace("l-1", "").replace("l1", "")
    return pageNbr
    

def removeYears(pageNbr):
    l = re.findall(r'\d{4}', pageNbr)
    if len(l)!=0:
        for s in l:
            pageNbr = pageNbr.replace(s, '')
    return pageNbr


def getPagesWhenRange(pageNbr):
    pgs = re.search(r'\d+\s*-\s*\d+', pageNbr).group().split("-")
    start = int(pgs[0])-1
    end = int(pgs[1])
    range_nbr = list(range(start, end))
    others = re.findall(r'\d+', pageNbr)
    others = [int(i)-1 for i in others]
    all_pgs = range_nbr + others
    return list(set(all_pgs))
    

def cleanPageNbr(pageNbr):
    """
    :param pageNbr: non-standard text introduced by analyst
    :return: list of string : cleaned page numbers
    """
    pageNbr = str(pageNbr)
    pageNbr = removeFakePgNbrs(pageNbr)
    pageNbr = removeYears(pageNbr)
    if re.findall(r'\d+\s*-\s*\d+', pageNbr):
        pages = getPagesWhenRange(pageNbr)
        return pages 
    pages = re.findall(r'\d+', pageNbr)
    pages = [int(i)-1 for i in pages if i!= 0]
    return list(set(pages))

#df_T5_100['answerPage'] = df_T5_100.apply(lambda x: cleanPageNbr(x['analystSource']), axis=1)
df_bm25_full['answerPage'] = df_bm25_full.apply(lambda x: cleanPageNbr(x['analystSource']), axis=1)

In [10]:
#df_electra = df_T5_100.copy()
df_electra = df_bm25_full.copy()
df_electra

,index,idOption,idQuestionUnique,textOption,optionHelp,textQuestion,idQuestionType,optionCode,idQuestion,questionCode,...,certificateQuest,answerPage,query,query_type,query_id,queryWordCount,passages,bm25TopPassages,bm25posPassages,bm25negPassages
0,3,2342,300,Provision of fuel consumption reduction inform...,Examples of documents to attach: product/servi...,Does your company provide services to clients ...,1,ENV420_10,300,ENV420,...,0,[22],detailed information available on fuel consump...,option_help,0,24,"[[0, 0, —toward the toyota environmental chall...",[fcev: expanded and improved fuel cell stack a...,[zero co2 emissions challenge the lca that toy...,[fcev: expanded and improved fuel cell stack a...
1,4,9666,1202,Reduction of water consumption through employe...,Examples of documents to attach: training mate...,What actions are in place regarding water mana...,1,ENV321_10,28,ENV321,...,0,[50],company specific employee awareness programs g...,option_help,1,16,"[[0, 406, —toward the toyota environmental cha...",[ensure that all global employees are aware of...,[with its suppliers. we continued these effort...,[ensure that all global employees are aware of...
2,5,9667,1202,Reduction of water consumption through innovat...,Examples of documents to attach: standard oper...,What actions are in place regarding water mana...,1,ENV321_20,28,ENV321,...,0,[31],implementation of innovative equipment to enab...,option_help,2,8,"[[0, 812, —toward the toyota environmental cha...",[region • contribute to air quality improvemen...,[editorial policy contents overview highlights...,[region • contribute to air quality improvemen...
3,6,10514,1300,Integration of ecodesign features in product d...,Examples of documents to attach: design specif...,What actions are in place to reduce the enviro...,1,ENV386_01,294,ENV386,...,0,[20],design of the product with special considerati...,option_help,3,29,"[[0, 1218, —toward the toyota environmental ch...",[editorial policy contents overview highlights...,[to curb global warming by reducing co2 emissi...,[editorial policy contents overview highlights...
4,7,10515,1300,Environmental impact analyses performed (e.g. ...,Examples of documents to attach: carbon footpr...,What actions are in place to reduce the enviro...,1,ENV386_02,294,ENV386,...,0,[12],environmental impact analysis of the product d...,option_help,4,51,"[[0, 1624, —toward the toyota environmental ch...",[goals: international goals for the period fro...,[editorial policy contents overview highlights...,[goals: international goals for the period fro...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8345,32066,18512,2348,Health care coverage of employees in place,Examples of documents to attach: detailed heal...,What actions are in place regarding working co...,1,LAB3201_6,1737,LAB3201,...,0,[4],detailed coverage of company's specific health...,option_help,8345,24,"[[0, 2445552, 1 employee handbook 2019], [1, 2...",[group health insurance coverage if they lose ...,[from the employee’s paycheck before taxes are...,[group health insurance coverage if they lose ...
8346,32067,18509,2348,Additional leave beyond standard vacation days,Examples of documents to attach: employee hand...,What actions are in place regarding working co...,1,LAB3201_3,1737,LAB3201,...,0,"[2, 3]",provision of additional leave beyond standard ...,option_help,8346,22,"[[0, 2445724, handbook employee], [1, 2445725,...",[to the end of the current pay period. the pay...,[to the end of the current pay period. the pay...,"[handbook employee, revised 6.28.19 2 family m..."
8347,32069,18510,2348,Flexible organization of work available to emp...,Examples of documents to attach: standard oper...,What actions are in place regarding working co...,1,LAB3201_4,1737,LAB3201,...,0,[0],provision of flexible working arrangement (eg ...,option_help,8347,33,"[[0, 2445739, page 1 ypsomed holding ag brunnm...",[page 2 about the ypsomed group the ypsomed gr.

### Training dataset creation

In [45]:
df_electra.to_csv('df_electra.csv', index=False)

In [ ]:
# Creating dataset with all possible training examples
rows_list = []
for row in range(len(df_electra)):
#    for i in range(len(df_electra.T5posPassages.iloc[row])):
    for i in range(len(df_electra.bm25posPassages.iloc[row])):
        dict_example = {}
        dict_example.update({'query': df_electra['query'].iloc[row]}) 
        dict_example.update({'query_id': df_electra['query_id'].iloc[row]}) 
        dict_example.update({'answerPage': df_electra['answerPage'].iloc[row]}) 
#        dict_example.update({'passage': df_electra['T5posPassages'].iloc[row][i][3]})
        dict_example.update({'passage': df_electra['bm25posPassages'].iloc[row][i]})
        dict_example.update({'label': 1}) 

        rows_list.append(dict_example)
#    for i in range(len(df_electra.T5negPassages.iloc[row])):
    for i in range(len(df_electra.bm25negPassages.iloc[row])):
        dict_example = {}
        dict_example.update({'query': df_electra['query'].iloc[row]}) 
        dict_example.update({'query_id': df_electra['query_id'].iloc[row]}) 
        dict_example.update({'answerPage': df_electra['answerPage'].iloc[row]}) 
#        dict_example.update({'passage': df_electra['T5negPassages'].iloc[row][i][3]})
        dict_example.update({'passage': df_electra['bm25negPassages'].iloc[row][i]})
        dict_example.update({'label': 0}) 

        rows_list.append(dict_example)

df_binary_examples_full = pd.DataFrame(rows_list, columns = ['query', 'query_id', 'answerPage', 'passage', 'label'])      
df_binary_examples_full['query_pass'] = df_binary_examples_full.apply(lambda x: [x['query'], x['passage']], axis=1)

In [ ]:
df_binary_examples_full

In [ ]:
df_binary_examples_full.label.value_counts()

In [35]:
df_binary_examples_full.to_csv('df_binary_examples_full.csv', index=False)

In [36]:
# Creating training and test sets
df_binary_examples_full_query_train = df_binary_examples_full[df_binary_examples_full['query_id'].isin(train_query_ids)]
df_binary_examples_full_query_test = df_binary_examples_full[df_binary_examples_full['query_id'].isin(test_query_ids)]

df_binary_examples_full_supplier_train = df_binary_examples_full[df_binary_examples_full['query_id'].isin(train_supplierIds_queryIds)] 
df_binary_examples_full_supplier_test = df_binary_examples_full[df_binary_examples_full['query_id'].isin(test_supplierIds_queryIds)] 

df_binary_examples_full_query_train.to_csv('df_binary_examples_full_query_train.csv', index=False)
df_binary_examples_full_query_test.to_csv('df_binary_examples_full_query_test.csv', index=False)

df_binary_examples_full_supplier_train.to_csv('df_binary_examples_full_supplier_train.csv', index=False)
df_binary_examples_full_supplier_test.to_csv('df_binary_examples_full_supplier_test.csv', index=False)

In [37]:
# Creating a smaller more balanced dataset with only up to 10 top positives and 10 negatives per query

import random 

rows_list = []
count = 0
for row in range(len(df_electra)):
#    for i in range(min(10,len(df_electra.T5posPassages.iloc[row]))):
    for i in range(min(10,len(df_electra.bm25posPassages.iloc[row]))):
        dict_example = {}
        dict_example.update({'query': df_electra['query'].iloc[row]}) 
        dict_example.update({'query_id': df_electra['query_id'].iloc[row]}) 
        dict_example.update({'answerPage': df_electra['answerPage'].iloc[row]}) 
#        dict_example.update({'passage': df_electra['T5posPassages'].iloc[row][i][3]})
        dict_example.update({'passage': df_electra['bm25posPassages'].iloc[row][i]})
        dict_example.update({'label': 1}) 
        dict_example.update({'example_id': count}) 
        count += 1
        rows_list.append(dict_example)
    
#    neg_samples_idx = random.sample(range(len(df_electra.T5negPassages.iloc[row])), min(10, len(df_electra.T5negPassages.iloc[row])))
    neg_samples_idx = random.sample(range(len(df_electra.bm25negPassages.iloc[row])), min(10, len(df_electra.bm25negPassages.iloc[row])))
    for i in neg_samples_idx:
        dict_example = {}
        dict_example.update({'query': df_electra['query'].iloc[row]}) 
        dict_example.update({'query_id': df_electra['query_id'].iloc[row]}) 
        dict_example.update({'answerPage': df_electra['answerPage'].iloc[row]}) 
#        dict_example.update({'passage': df_electra['T5negPassages'].iloc[row][i][3]})
        dict_example.update({'passage': df_electra['bm25negPassages'].iloc[row][i]})
        dict_example.update({'label': 0}) 
        dict_example.update({'example_id': count}) 
        count += 1
        rows_list.append(dict_example)

df_binary_examples_small = pd.DataFrame(rows_list, columns = ['query', 'query_id', 'answerPage', 'passage', 'label', 'example_id'])      
df_binary_examples_small['query_pass'] = df_binary_examples_small.apply(lambda x: [x['query'], x['passage']], axis=1)
df_binary_examples_small

,query,query_id,answerPage,passage,label,example_id,query_pass
0,detailed information available on fuel consump...,0,[22],zero co2 emissions challenge the lca that toyo...,1,0,[detailed information available on fuel consum...
1,detailed information available on fuel consump...,0,[22],editorial policy contents overview highlights ...,1,1,[detailed information available on fuel consum...
2,detailed information available on fuel consump...,0,[22],"public facilities, plants, and so on. as a par...",0,2,[detailed information available on fuel consum...
3,detailed information available on fuel consump...,0,[22],with nature environmental management fy 2015 2...,0,3,[detailed information available on fuel consum...
4,detailed information available on fuel consump...,0,[22],editorial policy contents overview highlights ...,0,4,[detailed information available on fuel consum...
...,...,...,...,...,...,...,...
117249,monetary remuneration for employees beyond bas...,8349,[0],again committed to link reward more closely wi...,1,117249,[monetary remuneration for employees beyond ba...
117250,monetary remuneration for employees beyond bas...,8349,[0],2019 stvrs gkn powder metallurgy may 2019 2019...,1,117250,[monetary remuneration for employees beyond ba...
117251,monetary remuneration for employees beyond bas...,8349,[0],be more agile and individually. to deliver our...,0,117251,[monetary remuneration for employees beyond ba...
117252,monetary remuneration for employees beyond bas...,8349,[0],year there will be an equal weighting between ...,0,117252,[monetary remuneration for employees beyond ba...


In [38]:
df_binary_examples_small.label.value_counts()

0    78848
1    38406
Name: label, dtype: int64

In [39]:
df_binary_examples_small.to_csv('df_binary_examples_small.csv', index=False)

In [40]:
# Creating training and test sets based on query_ids and supplier_ids
df_binary_examples_small_query_train = df_binary_examples_small[df_binary_examples_small['query_id'].isin(train_query_ids)]
df_binary_examples_small_query_test = df_binary_examples_small[df_binary_examples_small['query_id'].isin(test_query_ids)]

df_binary_examples_small_supplier_train = df_binary_examples_small[df_binary_examples_small['query_id'].isin(train_supplierIds_queryIds)] 
df_binary_examples_small_supplier_test = df_binary_examples_small[df_binary_examples_small['query_id'].isin(test_supplierIds_queryIds)] 

df_binary_examples_small_query_train.to_csv('df_binary_examples_small_query_train.csv', index=False)
df_binary_examples_small_query_test.to_csv('df_binary_examples_small_query_test.csv', index=False)

df_binary_examples_small_supplier_train.to_csv('df_binary_examples_small_supplier_train.csv', index=False)
df_binary_examples_small_supplier_test.to_csv('df_binary_examples_small_supplier_test.csv', index=False)

### ELECTRA Reranker build and train - binary classifier

In [79]:
import torch
from torch.utils.data import TensorDataset, random_split, \
                            DataLoader, RandomSampler, SequentialSampler
from transformers import ElectraForSequenceClassification, ElectraTokenizer, \
                         AdamW, get_linear_schedule_with_warmup

epochs = 6
batch_size = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

elec_tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')

In [81]:
#text = df_binary_examples_small['query_pass'].tolist()
#labels = df_binary_examples_small['label'].tolist()
text_train = df_binary_examples_small_query_train['query_pass'].tolist()
labels_train = df_binary_examples_small_query_train['label'].tolist()

text_test = df_binary_examples_small_query_test['query_pass'].tolist()
labels_test = df_binary_examples_small_query_test['label'].tolist()

In [98]:
from sklearn.model_selection import train_test_split

def preprocess(raw_text, labels=None):
    encoded_batch = elec_tokenizer.batch_encode_plus(raw_text,
                                                truncation=True,
                                                add_special_tokens=True,
                                                padding='max_length',
                                                max_length = 256,
#                                                padding='longest',
                                                return_attention_mask=True,
                                                return_tensors = 'pt')
    if labels:
        labels = torch.tensor(labels)
        return encoded_batch['input_ids'], encoded_batch['attention_mask'], labels
    return encoded_batch['input_ids'], encoded_batch['attention_mask']

def format_time(elapsed):
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    return str(datetime.timedelta(seconds=elapsed_rounded))

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


# Split train-validation-test
#train_inputs, test_inputs, train_labels, test_labels = train_test_split(text, labels,
#                                                            random_state=42, test_size=0.2)

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(text_train, labels_train,
                                                            random_state=42, test_size=0.2)

test_inputs, test_labels = text_test, labels_test

In [99]:
test_input_ids, test_attention_mask, test_labels = preprocess(test_inputs, test_labels)
# Combine the validation inputs into a TensorDataset
test_dataset = TensorDataset(
    test_input_ids,
    test_attention_mask,
    test_labels)

test_dataloader = DataLoader(
            test_dataset,
            sampler = SequentialSampler(test_dataset),
            batch_size = batch_size)

In [ ]:
train_input_ids, train_attention_mask, train_labels = preprocess(train_inputs, train_labels)
# Combine the training inputs into a TensorDataset
train_dataset = TensorDataset(
    train_input_ids,
    train_attention_mask,
    train_labels)

val_input_ids, val_attention_mask, validation_labels = preprocess(validation_inputs, validation_labels)
# Combine the validation inputs into a TensorDataset
validation_dataset = TensorDataset(
    val_input_ids,
    val_attention_mask,
    validation_labels)

test_input_ids, test_attention_mask, test_labels = preprocess(test_inputs, test_labels)
# Combine the validation inputs into a TensorDataset
test_dataset = TensorDataset(
    test_input_ids,
    test_attention_mask,
    test_labels)

In [95]:
# Saving TensorDatasets to avoid future recompute
torch.save(train_dataset, 'train_dataset_small')
torch.save(validation_dataset, 'validation_dataset_small')
torch.save(test_dataset, 'test_dataset_small')

In [96]:
# Loading saved TensorDatasets to save compute time
#train_dataset = torch.load('train_dataset_small')
#validation_dataset = torch.load('validation_dataset_small')
#test_dataset = torch.load('test_dataset_small')

In [ ]:
# Create the DataLoaders
train_dataloader = DataLoader(
            train_dataset,
            sampler = SequentialSampler(train_dataset),
            batch_size = batch_size)

validation_dataloader = DataLoader(
            validation_dataset,
            sampler = SequentialSampler(validation_dataset),
            batch_size = batch_size)

test_dataloader = DataLoader(
            test_dataset,
            sampler = SequentialSampler(test_dataset),
            batch_size = batch_size)

In [ ]:
try:
    state_dict = torch.load("Electra_bin_6e.pt")
    print("Loading trained model...")
    model = ElectraForSequenceClassification.from_pretrained('google/electra-small-discriminator',
    state_dict=state_dict)
    print("Trained model loaded!")
except Exception as e:
    print("Unable to load trained model.")
    print(e)
    model = ElectraForSequenceClassification.from_pretrained('google/electra-small-discriminator',
        num_labels = 2)

model.to(device) 

In [ ]:
#Training parameters
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # Learning Rate - Default is 5e-5
                  eps = 1e-8 # Adam Epsilon  - Default is 1e-8.
                )


# Training loop
training_stats = []
validation_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# Total number of training steps is [number of batches] x [number of epochs]
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

# This variable will evaluate the convergence on the training
consecutive_epochs_with_no_improve = 0

In [ ]:
# Training & Validation
for epoch in range(0, epochs):
    print("")
    print(f'########## Epoch {epoch+1} / {epochs} ##########')
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode
    model.train()

    # For each batch of training data
    for step, batch in enumerate(train_dataloader):
        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = time.time() - t0

            # Report progress
            print(f'  Batch {step}  of  {len(train_dataloader)}    Elapsed: {format_time(elapsed)}.')

        # Unpack this training batch from our dataloader. 
        input_id = batch[0].to(device)
        attention_mask = batch[1].to(device)
        label = batch[2].to(device)

        # Clear any previously calculated gradients before performing a backward pass
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch)
        loss, logits = model(input_id, 
                              token_type_ids=None, 
                              attention_mask=attention_mask, 
                              labels=label)

        # Accumulate the training loss over all of the batches
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients
        loss.backward()

        # Clip the norm of the gradients to 1.0
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches
    avg_train_loss = total_train_loss / len(train_dataloader)   

    if epoch > 0:
        if min([stat['Training Loss'] for stat in training_stats]) <= avg_train_loss:
            # i.e. If there is not improvement
            consecutive_epochs_with_no_improve += 1
        else:
            # If there is improvement
            consecutive_epochs_with_no_improve = 0
            print("Model saved!")
            torch.save(model.state_dict(), "Electra_bin_6e.pt")

    # Measure how long this epoch took
    training_time = time.time() - t0

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))

    # Record all statistics from this epoch.
    training_stats.append(
      {
          'epoch': epoch + 1,
          'Training Loss': avg_train_loss,
          'Training Time': training_time,
      }
    )

    print("epoch: {} \n".format(epoch + 1), file=outfile)
    print("Training Loss: {} \n".format(avg_train_loss), file=outfile)
    print("Training Time: {} \n".format(training_time), file=outfile)

    if consecutive_epochs_with_no_improve == 2:
        print("Stop training : The loss has not changed since 2 epochs!")
        break


    # Tracking variables for validation
    eval_loss, eval_accuracy = 0, 0
    batch_count = 0

    t1 = time.time()
    # Validation of the model
    model.eval()
    print('Evaluation...')

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        elapsed = time.time() - t1
        print(f'  Batch {batch_count}  of  {len(validation_dataloader)}    Elapsed: {format_time(elapsed)}.')

        # Unpack this training batch from our dataloader. 
        input_id = batch[0].to(device)
        attention_mask = batch[1].to(device)
        label = batch[2].to(device)

        with torch.no_grad():
        # Forward pass, calculate logit predictions
            outputs =  model(input_id, token_type_ids=None, 
                            attention_mask=attention_mask,
                            labels=label)
            loss, logits = outputs[:2]

        # Accumulate the training loss over all of the batches
        eval_loss += loss.item()

        # Move logits and labels to CPU if GPU is used
        logits = logits.detach().cpu().numpy()
        label_ids = label.to('cpu').numpy()
        predictions = np.argmax(logits, axis=1).flatten()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)

        eval_accuracy += tmp_eval_accuracy
        if batch_count == 0:
            global_predictions = predictions
        else:
            global_predictions = np.append(global_predictions,predictions)
        batch_count += 1


    avg_validation_loss = eval_loss / len(validation_dataloader)   


    # Measure how long this epoch took
    validation_time = time.time() - t1

    print("")
    print("  Average validation loss: {0:.2f}".format(avg_validation_loss))
    print("  Validation took: {:}".format(validation_time))
    print("  Validation Accuracy: {}".format(eval_accuracy/batch_count))
    # Calculate the average loss over all of the batches

    # Record all statistics from this epoch.
    validation_stats.append(
      {
          'Training Loss': avg_validation_loss,
          'Training Time': validation_time,
      }
    )

    print("Validation Loss: {} \n".format(avg_validation_loss), file=outfile)
    print("Validation Time: {} \n".format(validation_time), file=outfile)
        
    

torch.save(model.state_dict(), "Electra_bin_6e.pt")
print("Final Model saved!")

In [ ]:
#Evaluating on dev set
print("Test mode: \n")

t2 = time.time()
model.eval()
batch_count = 0
test_loss, test_accuracy = 0, 0

for batch in test_dataloader:
        
    # Progress update every 40 batches.
    if batch_count % 40 == 0 and not batch_count == 0:
        # Calculate elapsed time in minutes.
        elapsed = time.time() - t2

        # Report progress
        print(f'  Batch {batch_count}  of  {len(test_dataloader)}    Elapsed: {format_time(elapsed)}.')
        
    input_id = batch[0].to(device)
    attention_mask = batch[1].to(device)
    label = batch[2].to(device)

    with torch.no_grad():
    # Forward pass, calculate logit predictions
        outputs =  model(input_id, token_type_ids=None, 
                              attention_mask=attention_mask,
                              labels=label)
        loss, logits = outputs[:2]
        probs = F.softmax(logits, dim=1)

    test_loss += loss.item()

    # Move logits and labels to CPU if GPU is used
    logits = logits.detach().cpu().numpy()
    probs = probs.detach().cpu().numpy()
    label_ids = label.to('cpu').numpy()
 
    probs_list.extend([i for i in probs])
    predictions = np.argmax(logits, axis=1).flatten()

    tmp_test_accuracy = flat_accuracy(logits, label_ids)

    test_accuracy += tmp_test_accuracy
    if batch_count == 0:
        global_predictions = predictions
    else:
        global_predictions = np.append(global_predictions,predictions)
    batch_count += 1


In [ ]:
avg_test_loss = test_loss / len(validation_dataloader)   

test_time = time.time() - t2

global_preds = torch.from_numpy(global_predictions)

report = metrics.classification_report(test_labels, global_preds, zero_division=0)

confusion_matrix = metrics.confusion_matrix(test_labels, global_preds)

print("")
print("  Average test loss: {0:.2f}".format(avg_test_loss))
print("  Test took: {:}".format(test_time))
print("  Test Accuracy: {}".format(test_accuracy/batch_count))
print("")
print("Report on test set: {} \n".format(report))
print("Confusion Matrix on test set: \n{} \n".format(confusion_matrix))

In [ ]:
sns.heatmap(confusion_matrix, annot=True)

In [ ]:
# Saving prediction results for future evaluation
df_binary_test = df_binary_examples_small_query_test.copy()

df_binary_test['probs'] = probs_list
df_binary_test['prediction'] = global_predictions

df_electra = pd.read_csv('df_electra.csv')
df_electra['passages'] = df_electra['passages'].apply(lambda x: ast.literal_eval(x))
passages_dict = {i[2]:i[0] for p in df_electra['passages'].tolist() for i in p}

df_binary_test['pass_page'] = df_binary_test['passage'].apply(lambda x: passages_dict[x])

df_binary_test['page_pred_score_pas'] = df_binary_test.apply(lambda x: [x['pass_page'], x['prediction'], x['probs'], x['passage']], axis=1)

df_binary_test.to_csv('df_binary_pred_test.csv', index=False)

### Electra binary classifier evaluation

In [249]:
# Reimporting the test dataset with results of Electra predictions to evaluate the model
import ast, re
df_binary_pred_test = pd.read_csv('df_binary_pred_test.csv')
#df_binary_pred_test = pd.read_csv('df_binary_suppliers_pred_test.csv')
df_binary_pred_test['probs'] = df_binary_pred_test['probs'].apply(lambda x: [t for t in str(x).strip('][').split(' ') if t])
df_binary_pred_test['answerPage'] = df_binary_pred_test['answerPage'].apply(lambda x: re.findall(r'\d+', x))
df_binary_pred_test['page_label_pred_score_pas'] = df_binary_pred_test.apply(lambda x: [x['pass_page'], x['label'], x['prediction'], x['probs'][1], x['passage']], axis=1)
df_binary_pred_test

,query,query_id,answerPage,passage,label,example_id,query_pass,probs,prediction,pass_page,page_pred_score_pas,page_label_pred_score_pas
0,detailed information on take back or recycling...,75,[18],release liner recycling _ we help customers to...,1,1008,['detailed information on take back or recycli...,"[-0.1789516, 0.13999867]",1,18,"[18, 1, array([-0.1789516 , 0.13999867], dtyp...","[18, 1, 1, 0.13999867, release liner recycling..."
1,detailed information on take back or recycling...,75,[18],“the little prince” _ nursery & ironing service,0,1009,['detailed information on take back or recycli...,"[1.1446745, -1.4892147]",0,36,"[36, 0, array([ 1.1446745, -1.4892147], dtype=...","[36, 0, 0, -1.4892147, “the little prince” _ n..."
2,detailed information on take back or recycling...,75,[18],energy saving efforts,0,1010,['detailed information on take back or recycli...,"[1.2210922, -1.5654072]",0,4,"[4, 0, array([ 1.2210922, -1.5654072], dtype=f...","[4, 0, 0, -1.5654072, energy saving efforts]"
3,detailed information on take back or recycling...,75,[18],focus on illness & injury prevention,0,1011,['detailed information on take back or recycli...,"[1.4576044, -1.8454094]",0,33,"[33, 0, array([ 1.4576044, -1.8454094], dtype=...","[33, 0, 0, -1.8454094, focus on illness & inju..."
4,detailed information on take back or recycling...,75,[18],"enjoy nature originators of the 4,1 ha “peace ...",0,1012,['detailed information on take back or recycli...,"[1.3412256, -1.6995752]",0,42,"[42, 0, array([ 1.3412256, -1.6995752], dtype=...","[42, 0, 0, -1.6995752, enjoy nature originator..."
...,...,...,...,...,...,...,...,...,...,...,...,...
24214,provision of flexible working arrangement (eg ...,8347,[0],page 2 about the ypsomed group the ypsomed gro...,0,109844,['provision of flexible working arrangement (e...,"[1.616275, -2.023004]",0,1,"[1, 0, array([ 1.616275, -2.023004], dtype=flo...","[1, 0, 0, -2.023004, page 2 about the ypsomed ..."
24215,detailed coverage of company's specific health...,8348,[1],platform is designed to attract and retain mot...,1,109845,"[""detailed coverage of company's specific heal...","[-0.33545083, 0.3508757]",1,1,"[1, 1, array([-0.33545083, 0.3508757 ], dtype...","[1, 1, 1, 0.3508757, platform is designed to a..."
24216,detailed coverage of company's specific health...,8348,[1],revised 6.28.19 2 section iii – benefits tekno...,1,109846,"[""detailed coverage of company's specific heal...","[-0.6488942, 0.7226348]",1,1,"[1, 1, array([-0.6488942, 0.7226348], dtype=f...","[1, 1, 1, 0.7226348, revised 6.28.19 2 section..."
24217,detailed coverage of company's specific health...,8348,[1],employee savings with employer contributions a...,1,109847,"[""detailed coverage of company's specific heal...","[-0.58144724, 0.6184851]",1,1,"[1, 1, array([-0.58144724, 0.6184851 ], dtype...","[1, 1, 1, 0.6184851, employee savings with emp..."


In [251]:
df_binary_pred_test.drop_duplicates(subset ="query_pass",
                     keep = "first", inplace = True)
df_binary_pred_test

,query,query_id,answerPage,passage,label,example_id,query_pass,probs,prediction,pass_page,page_pred_score_pas,page_label_pred_score_pas
0,detailed information on take back or recycling...,75,[18],release liner recycling _ we help customers to...,1,1008,['detailed information on take back or recycli...,"[-0.1789516, 0.13999867]",1,18,"[18, 1, array([-0.1789516 , 0.13999867], dtyp...","[18, 1, 1, 0.13999867, release liner recycling..."
1,detailed information on take back or recycling...,75,[18],“the little prince” _ nursery & ironing service,0,1009,['detailed information on take back or recycli...,"[1.1446745, -1.4892147]",0,36,"[36, 0, array([ 1.1446745, -1.4892147], dtype=...","[36, 0, 0, -1.4892147, “the little prince” _ n..."
2,detailed information on take back or recycling...,75,[18],energy saving efforts,0,1010,['detailed information on take back or recycli...,"[1.2210922, -1.5654072]",0,4,"[4, 0, array([ 1.2210922, -1.5654072], dtype=f...","[4, 0, 0, -1.5654072, energy saving efforts]"
3,detailed information on take back or recycling...,75,[18],focus on illness & injury prevention,0,1011,['detailed information on take back or recycli...,"[1.4576044, -1.8454094]",0,33,"[33, 0, array([ 1.4576044, -1.8454094], dtype=...","[33, 0, 0, -1.8454094, focus on illness & inju..."
4,detailed information on take back or recycling...,75,[18],"enjoy nature originators of the 4,1 ha “peace ...",0,1012,['detailed information on take back or recycli...,"[1.3412256, -1.6995752]",0,42,"[42, 0, array([ 1.3412256, -1.6995752], dtype=...","[42, 0, 0, -1.6995752, enjoy nature originator..."
...,...,...,...,...,...,...,...,...,...,...,...,...
24214,provision of flexible working arrangement (eg ...,8347,[0],page 2 about the ypsomed group the ypsomed gro...,0,109844,['provision of flexible working arrangement (e...,"[1.616275, -2.023004]",0,1,"[1, 0, array([ 1.616275, -2.023004], dtype=flo...","[1, 0, 0, -2.023004, page 2 about the ypsomed ..."
24215,detailed coverage of company's specific health...,8348,[1],platform is designed to attract and retain mot...,1,109845,"[""detailed coverage of company's specific heal...","[-0.33545083, 0.3508757]",1,1,"[1, 1, array([-0.33545083, 0.3508757 ], dtype...","[1, 1, 1, 0.3508757, platform is designed to a..."
24216,detailed coverage of company's specific health...,8348,[1],revised 6.28.19 2 section iii – benefits tekno...,1,109846,"[""detailed coverage of company's specific heal...","[-0.6488942, 0.7226348]",1,1,"[1, 1, array([-0.6488942, 0.7226348], dtype=f...","[1, 1, 1, 0.7226348, revised 6.28.19 2 section..."
24217,detailed coverage of company's specific health...,8348,[1],employee savings with employer contributions a...,1,109847,"[""detailed coverage of company's specific heal...","[-0.58144724, 0.6184851]",1,1,"[1, 1, array([-0.58144724, 0.6184851 ], dtype...","[1, 1, 1, 0.6184851, employee savings with emp..."


In [252]:
grouped = df_binary_pred_test.groupby('query_id')['page_label_pred_score_pas', 'answerPage']
df_binary_pred_final = grouped.aggregate(lambda x: list(x))
df_binary_pred_final

/anaconda/envs/pygaggle2/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,page_label_pred_score_pas,answerPage
query_id,,
75,"[[18, 1, 1, 0.13999867, release liner recyclin...","[[18], [18], [18], [18], [18], [18], [18], [18..."
76,"[[14, 1, 0, -0.2528201, sensitization programs...","[[14], [14], [14], [14], [14], [14], [14], [14..."
77,"[[13, 1, 0, -0.25707737, rainwater collector i...","[[13], [13], [13], [13], [13], [13], [13], [13..."
78,"[[10, 1, 1, 1.0426259, invest in less energy c...","[[10], [10], [10], [10], [10], [10], [10], [10..."
79,"[[17, 1, 1, 0.71462715, shredder / collector f...","[[17], [17], [17], [17], [17], [17], [17], [17..."
...,...,...
8318,"[[1, 1, 1, 1.0309529, important information ab...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ..."
8337,"[[11, 1, 1, 0.55018497, 2019 yer process . dl ...","[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13..."
8346,"[[3, 1, 1, 0.75872767, to the end of the curre...","[[2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3..."


In [253]:
def sort(sub_li):  
    sub_li.sort(key = lambda x: x[3], reverse = True)
    return sub_li

df_binary_pred_final['page_label_pred_score_pas'] = df_binary_pred_final['page_label_pred_score_pas'].apply(lambda x: sort(x))
df_binary_pred_final['page_label_pred_score_pas'].iloc[0]

[[18,
  1,
  1,
  '0.13999867',
  'release liner recycling _ we help customers to recycle liner waste we also invite all our customers to join our release liner recycling program. together we intend to yearly remove an extra 150 to 200 tons of pure release liner from the waste stream.'],
 [33, 0, 0, '-1.8454094', 'focus on illness & injury prevention'],
 [32, 0, 0, '-1.7776856', 'live healthy'],
 [60,
  0,
  0,
  '-1.7665367',
  'support local social organisations cakes, sandwiches, snacks & bio-soup are ordered from “de vliering”, a local workshop, run by mentally handicapped adults.'],
 [40,
  0,
  0,
  '-1.7157924',
  'free fresh fruit on monday monday is “fresh fruit day” in boechout. a local grower supplies us fresh apples from his own “integrated culture”; he deploys useful insects as pest control rather than insecticides. these apples are supplemented with various fresh “fruit of the week”.'],
 [42,
  0,
  0,
  '-1.6995752',
  'enjoy nature originators of the 4,1 ha “peace fores

In [254]:
# Scoring success on top k results
def predict_pages_electra(k, top_passages, answer_page):
    prediction = 0
    for j in range(k):
        try:
            if str(top_passages[j][0]) in answer_page[0]:
                prediction = 1 
        except:
            pass
    return prediction

# Running the evaluation on all lines except the ones with 1 page only
print('Result on query_id test set:')
for k in [1,3,5,10]:
    results = df_binary_pred_final.apply(lambda x: predict_pages_electra(k, x['page_label_pred_score_pas'], x['answerPage']), axis=1)
    try:
        print("Success @ {}: {:f}".format(k, sum(results)/len(results)))
    except:
        print('Error, try lower k')

Result on query_id test set:
Success @ 1: 0.666851
Success @ 3: 0.765357
Success @ 5: 0.791367
Success @ 10: 0.850581


In [255]:
# Investigating the distribution of Success@3 depending on the type of report
import itertools
import numpy as np

elec_query_ids_pred = list(df_binary_pred_final.index.values)
df_binary_pred_final2 = df_binary_pred_final.copy()
df_binary_pred_final2['typeName'] = df_electra[df_electra['query_id'].isin(elec_query_ids_pred)]['typeName']
df_binary_pred_final2['numPages'] = df_electra[df_electra['query_id'].isin(elec_query_ids_pred)]['numPages']
df_binary_pred_final2 = df_binary_pred_final2.reset_index()

grouped_type_elec = df_binary_pred_final2.groupby('typeName')['query_id', 'answerPage', 'numPages', 'page_label_pred_score_pas']
df_elec_full_type = grouped_type_elec.aggregate(lambda x: list(x))
df_elec_full_type = df_elec_full_type.reset_index()
df_elec_full_type['numPages'] = df_elec_full_type['numPages'].apply(lambda x: np.mean(x))
#df_bm25_full_type['query_id'] = df_bm25_full_type['query_id'].apply(lambda x: int(str(list(set(x))).strip('[]')))
df_elec_full_type['answerPage'] = df_elec_full_type['answerPage'].apply(lambda x: [i for i,_ in itertools.groupby(x)])
df_elec_full_type

elec_types = df_elec_full_type['typeName'].tolist()
print('Success @ 3 for type:')
for s in elec_types:
    index_s_elec = elec_types.index(s)
    query_idx_elec = df_elec_full_type.query_id.iloc[elec_types.index(s)]
    df_types_temp_elec = df_binary_pred_final2[df_binary_pred_final2['query_id'].isin(query_idx_elec)]
    results = df_types_temp_elec.apply(lambda x: predict_pages_electra(3, x['page_label_pred_score_pas'], x['answerPage']), axis=1)
    if len(results) != 0:
        print("  {} with average {:.0f} pages: {:f}".format(s, df_elec_full_type['numPages'].iloc[index_s_elec], sum(results)/len(results)))

Success @ 3 for type:
  Annual_Report with average 99 pages: 0.753012
  Audit_Report with average 21 pages: 0.647059
  CSR_Report with average 58 pages: 0.755208
  Certificate with average 16 pages: 0.600000
  Code_of_Ethics_Code_of_Conduct with average 24 pages: 0.813953
  Employee_Handbook with average 50 pages: 0.787234
  Letter_to_EcoVadis with average 82 pages: 0.571429
  Other with average 33 pages: 0.833333
  Policy with average 25 pages: 0.800000
  Procedure with average 16 pages: 0.621622
  Supplier_Code_of_Conduct with average 16 pages: 1.000000
  Webpage_Intranet_extract with average 6 pages: 0.884615


/anaconda/envs/pygaggle2/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  # This is added back by InteractiveShellApp.init_path()


In [256]:
# Scoring MRR on top k results
def eval_MRR_electra(k, top_passages, answer_page):
    prediction = 0
    stop = False
    for j in range(k):
        try:
            if str(top_passages[j][0]) in answer_page[0] and stop == False:
                prediction = 1/(j+1)
                stop = True
        except:
            pass
    return prediction

# Running the evaluation 
for k in [1,3,5,10]:
    results = df_binary_pred_final.apply(lambda x: eval_MRR_electra(k, x['page_label_pred_score_pas'], x['answerPage']), axis=1)
    try:
        print("MRR @ {}: {:f}".format(k, sum(results)/len(results)))
    except:
        print('Error, try lower k')

MRR @ 1: 0.666851
MRR @ 3: 0.712415
MRR @ 5: 0.718364
MRR @ 10: 0.725939


In [257]:
df_binary_pred_final

,page_label_pred_score_pas,answerPage
query_id,,
75,"[[18, 1, 1, 0.13999867, release liner recyclin...","[[18], [18], [18], [18], [18], [18], [18], [18..."
76,"[[24, 0, 0, -1.9604785, investments in presses...","[[14], [14], [14], [14], [14], [14], [14], [14..."
77,"[[33, 0, 0, -1.8165572, focus on illness & inj...","[[13], [13], [13], [13], [13], [13], [13], [13..."
78,"[[8, 0, 1, 1.1139504, save on energy consumpti...","[[10], [10], [10], [10], [10], [10], [10], [10..."
79,"[[17, 1, 1, 0.71462715, shredder / collector f...","[[17], [17], [17], [17], [17], [17], [17], [17..."
...,...,...
8318,"[[9, 0, 1, 1.1144593, collection and use, and ...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ..."
8337,"[[11, 1, 1, 0.55018497, 2019 yer process . dl ...","[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13..."
8346,"[[3, 1, 1, 1.1876292, vacation time. immediate...","[[2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3..."


In [258]:
# Investigating the distribution of Success@3 depending on the size of the documents

def page_range(x):
    if x < 31:
        pageRange = '2-30'
    elif x < 61:
        pageRange = '31-60'
    else:
        pageRange = '61-more'
    return pageRange

df_electra['pageRange'] = df_electra['numPages'].apply(lambda x: page_range(x))

import itertools

elec_query_ids_pred = list(df_binary_pred_final.index.values)
df_binary_pred_final2b = df_binary_pred_final.copy()
df_binary_pred_final2b['pageRange'] = df_electra[df_electra['query_id'].isin(elec_query_ids_pred)]['pageRange']
df_binary_pred_final2b = df_binary_pred_final2b.reset_index()

grouped_range_elec = df_binary_pred_final2b.groupby('pageRange')['query_id', 'answerPage', 'page_label_pred_score_pas']
df_elec_full_range = grouped_range_elec.aggregate(lambda x: list(x))
df_elec_full_range = df_elec_full_range.reset_index()
df_elec_full_range['answerPage'] = df_elec_full_range['answerPage'].apply(lambda x: [i for i,_ in itertools.groupby(x)])
df_elec_full_range

elec_ranges = df_elec_full_range['pageRange'].tolist()
print('Success @ 3 for range:')
for s in elec_ranges:
    index_s_elec = elec_ranges.index(s)
    query_idx_elec = df_elec_full_range.query_id.iloc[elec_ranges.index(s)]
    df_ranges_temp_elec = df_binary_pred_final2b[df_binary_pred_final2b['query_id'].isin(query_idx_elec)]
    results = df_ranges_temp_elec.apply(lambda x: predict_pages_electra(3, x['page_label_pred_score_pas'], x['answerPage']), axis=1)
    if len(results) != 0:
        print("  {}: {:f}".format(s, sum(results)/len(results)))

Success @ 3 for range:
  2-30: 0.811839
  31-60: 0.749591
  61-more: 0.748271


/anaconda/envs/pygaggle2/lib/python3.6/site-packages/ipykernel_launcher.py:21: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


### Using Electra predictions to create new training dataset

In [273]:
# First step is to use previously trained Electra model to generate predictions on each example of df_binary_examples_full
# Once generated via the above script applied to df_binary_examples_full, the predictions are stored in the following dataframe

df_binary_examples = df_binary_examples_full.copy()
df_binary_examples['probs'] = probs_list
df_binary_examples['prediction'] = global_predictions

df_electra = pd.read_csv('df_electra.csv')
df_electra['passages'] = df_electra['passages'].apply(lambda x: ast.literal_eval(x))
passages_dict = {i[2]:i[0] for p in df_electra['passages'].tolist() for i in p}

df_binary_examples['pass_page'] = df_binary_examples['passage'].apply(lambda x: passages_dict[x])

df_binary_examples['page_pred_score_pas'] = df_binary_examples.apply(lambda x: [x['pass_page'], x['prediction'], x['probs'], x['passage']], axis=1)

df_binary_examples.to_csv('df_binary_pred_full.csv', index=False)

In [107]:
# Second step is to import these predictions and use them to sort top positive and negative examples according to Electra
import ast
df_binary_pred_full = pd.read_csv('df_binary_pred_full.csv')
df_binary_pred_full['probs'] = df_binary_pred_full['probs'].apply(lambda x: [t for t in str(x).strip('][').split(' ') if t])
df_binary_pred_full['answerPage'] = df_binary_pred_full['answerPage'].apply(lambda x: re.findall(r'\d+', x))
df_binary_pred_full['page_label_pred_score_pas'] = df_binary_pred_full.apply(lambda x: [x['pass_page'], x['label'], x['prediction'], x['probs'][1], x['passage']], axis=1)
df_binary_pred_full

,query,query_id,answerPage,passage,label,query_pass,probs,prediction,pass_page,page_pred_score_pas,page_label_pred_score_pas
0,detailed information available on fuel consump...,0,[22],fcev: expanded and improved fuel cell stack an...,0,['detailed information available on fuel consu...,"[0.67176783, 0.32823217]",0,14,"[14, 0, array([0.67176783, 0.32823217], dtype=...","[14, 0, 0, 0.32823217, fcev: expanded and impr..."
1,detailed information available on fuel consump...,0,[22],"(eco-car and zev subsidies, reduced taxes) fos...",0,['detailed information available on fuel consu...,"[0.88445944, 0.1155405]",0,11,"[11, 0, array([0.88445944, 0.1155405 ], dtype=...","[11, 0, 0, 0.1155405, (eco-car and zev subsidi..."
2,detailed information available on fuel consump...,0,[22],"suppliers, energy providers, infrastructure de...",0,['detailed information available on fuel consu...,"[0.44180772, 0.5581923]",1,10,"[10, 1, array([0.44180772, 0.5581923 ], dtype=...","[10, 0, 1, 0.5581923, suppliers, energy provid..."
3,detailed information available on fuel consump...,0,[22],"award along with chubu electric power co., inc...",0,['detailed information available on fuel consu...,"[0.6354296, 0.3645704]",0,5,"[5, 0, array([0.6354296, 0.3645704], dtype=flo...","[5, 0, 0, 0.3645704, award along with chubu el..."
4,detailed information available on fuel consump...,0,[22],future society in harmony with nature environm...,0,['detailed information available on fuel consu...,"[0.70467395, 0.29532608]",0,21,"[21, 0, array([0.70467395, 0.29532608], dtype=...","[21, 0, 0, 0.29532608, future society in harmo..."
...,...,...,...,...,...,...,...,...,...,...,...
713017,monetary remuneration for employees beyond bas...,8349,[0],again committed to link reward more closely wi...,1,['monetary remuneration for employees beyond b...,"[0.1724616, 0.82753843]",1,0,"[0, 1, array([0.1724616 , 0.82753843], dtype=f...","[0, 1, 1, 0.82753843, again committed to link ..."
713018,monetary remuneration for employees beyond bas...,8349,[0],2019 stvrs gkn powder metallurgy may 2019 2019...,1,['monetary remuneration for employees beyond b...,"[0.50923276, 0.49076718]",0,0,"[0, 0, array([0.50923276, 0.49076718], dtype=f...","[0, 1, 0, 0.49076718, 2019 stvrs gkn powder me..."
713019,monetary remuneration for employees beyond bas...,8349,[0],2019 mip gkn powder metallurgy may 2019 2019 m...,0,['monetary remuneration for employees beyond b...,"[0.8720288, 0.12797113]",0,1,"[1, 0, array([0.8720288 , 0.12797113], dtype=f...","[1, 0, 0, 0.12797113, 2019 mip gkn powder meta..."
713020,monetary remuneration for employees beyond bas...,8349,[0],be more agile and individually. to deliver our...,0,['monetary remuneration for employees beyond b...,"[0.58132815, 0.41867188]",0,1,"[1, 0, array([0.58132815, 0.41867188], dtype=f...","[1, 0, 0, 0.41867188, be more agile and indivi..."


In [259]:
df_binary_pred_full.drop_duplicates(subset ="query_pass",
                     keep = "first", inplace = True)
df_binary_pred_full

,query,query_id,answerPage,passage,label,query_pass,probs,prediction,pass_page,page_pred_score_pas,page_label_pred_score_pas
0,detailed information available on fuel consump...,0,[22],fcev: expanded and improved fuel cell stack an...,0,['detailed information available on fuel consu...,"[0.67176783, 0.32823217]",0,14,"[14, 0, array([0.67176783, 0.32823217], dtype=...","[14, 0, 0, 0.32823217, fcev: expanded and impr..."
1,detailed information available on fuel consump...,0,[22],"(eco-car and zev subsidies, reduced taxes) fos...",0,['detailed information available on fuel consu...,"[0.88445944, 0.1155405]",0,11,"[11, 0, array([0.88445944, 0.1155405 ], dtype=...","[11, 0, 0, 0.1155405, (eco-car and zev subsidi..."
2,detailed information available on fuel consump...,0,[22],"suppliers, energy providers, infrastructure de...",0,['detailed information available on fuel consu...,"[0.44180772, 0.5581923]",1,10,"[10, 1, array([0.44180772, 0.5581923 ], dtype=...","[10, 0, 1, 0.5581923, suppliers, energy provid..."
3,detailed information available on fuel consump...,0,[22],"award along with chubu electric power co., inc...",0,['detailed information available on fuel consu...,"[0.6354296, 0.3645704]",0,5,"[5, 0, array([0.6354296, 0.3645704], dtype=flo...","[5, 0, 0, 0.3645704, award along with chubu el..."
4,detailed information available on fuel consump...,0,[22],future society in harmony with nature environm...,0,['detailed information available on fuel consu...,"[0.70467395, 0.29532608]",0,21,"[21, 0, array([0.70467395, 0.29532608], dtype=...","[21, 0, 0, 0.29532608, future society in harmo..."
...,...,...,...,...,...,...,...,...,...,...,...
713017,monetary remuneration for employees beyond bas...,8349,[0],again committed to link reward more closely wi...,1,['monetary remuneration for employees beyond b...,"[0.1724616, 0.82753843]",1,0,"[0, 1, array([0.1724616 , 0.82753843], dtype=f...","[0, 1, 1, 0.82753843, again committed to link ..."
713018,monetary remuneration for employees beyond bas...,8349,[0],2019 stvrs gkn powder metallurgy may 2019 2019...,1,['monetary remuneration for employees beyond b...,"[0.50923276, 0.49076718]",0,0,"[0, 0, array([0.50923276, 0.49076718], dtype=f...","[0, 1, 0, 0.49076718, 2019 stvrs gkn powder me..."
713019,monetary remuneration for employees beyond bas...,8349,[0],2019 mip gkn powder metallurgy may 2019 2019 m...,0,['monetary remuneration for employees beyond b...,"[0.8720288, 0.12797113]",0,1,"[1, 0, array([0.8720288 , 0.12797113], dtype=f...","[1, 0, 0, 0.12797113, 2019 mip gkn powder meta..."
713020,monetary remuneration for employees beyond bas...,8349,[0],be more agile and individually. to deliver our...,0,['monetary remuneration for employees beyond b...,"[0.58132815, 0.41867188]",0,1,"[1, 0, array([0.58132815, 0.41867188], dtype=f...","[1, 0, 0, 0.41867188, be more agile and indivi..."


In [314]:
df_binary_pred_full = df_binary_pred_full[df_binary_pred_full.astype(str)['answerPage'] != '[]']
df_binary_pred_full

,query,query_id,answerPage,passage,label,query_pass,probs,prediction,pass_page,page_pred_score_pas,page_label_pred_score_pas
0,detailed information available on fuel consump...,0,[22],fcev: expanded and improved fuel cell stack an...,0,['detailed information available on fuel consu...,"[0.67176783, 0.32823217]",0,14,"[14, 0, array([0.67176783, 0.32823217], dtype=...","[14, 0, 0, 0.32823217, fcev: expanded and impr..."
1,detailed information available on fuel consump...,0,[22],"(eco-car and zev subsidies, reduced taxes) fos...",0,['detailed information available on fuel consu...,"[0.88445944, 0.1155405]",0,11,"[11, 0, array([0.88445944, 0.1155405 ], dtype=...","[11, 0, 0, 0.1155405, (eco-car and zev subsidi..."
2,detailed information available on fuel consump...,0,[22],"suppliers, energy providers, infrastructure de...",0,['detailed information available on fuel consu...,"[0.44180772, 0.5581923]",1,10,"[10, 1, array([0.44180772, 0.5581923 ], dtype=...","[10, 0, 1, 0.5581923, suppliers, energy provid..."
3,detailed information available on fuel consump...,0,[22],"award along with chubu electric power co., inc...",0,['detailed information available on fuel consu...,"[0.6354296, 0.3645704]",0,5,"[5, 0, array([0.6354296, 0.3645704], dtype=flo...","[5, 0, 0, 0.3645704, award along with chubu el..."
4,detailed information available on fuel consump...,0,[22],future society in harmony with nature environm...,0,['detailed information available on fuel consu...,"[0.70467395, 0.29532608]",0,21,"[21, 0, array([0.70467395, 0.29532608], dtype=...","[21, 0, 0, 0.29532608, future society in harmo..."
...,...,...,...,...,...,...,...,...,...,...,...
713017,monetary remuneration for employees beyond bas...,8349,[0],again committed to link reward more closely wi...,1,['monetary remuneration for employees beyond b...,"[0.1724616, 0.82753843]",1,0,"[0, 1, array([0.1724616 , 0.82753843], dtype=f...","[0, 1, 1, 0.82753843, again committed to link ..."
713018,monetary remuneration for employees beyond bas...,8349,[0],2019 stvrs gkn powder metallurgy may 2019 2019...,1,['monetary remuneration for employees beyond b...,"[0.50923276, 0.49076718]",0,0,"[0, 0, array([0.50923276, 0.49076718], dtype=f...","[0, 1, 0, 0.49076718, 2019 stvrs gkn powder me..."
713019,monetary remuneration for employees beyond bas...,8349,[0],2019 mip gkn powder metallurgy may 2019 2019 m...,0,['monetary remuneration for employees beyond b...,"[0.8720288, 0.12797113]",0,1,"[1, 0, array([0.8720288 , 0.12797113], dtype=f...","[1, 0, 0, 0.12797113, 2019 mip gkn powder meta..."
713020,monetary remuneration for employees beyond bas...,8349,[0],be more agile and individually. to deliver our...,0,['monetary remuneration for employees beyond b...,"[0.58132815, 0.41867188]",0,1,"[1, 0, array([0.58132815, 0.41867188], dtype=f...","[1, 0, 0, 0.41867188, be more agile and indivi..."


In [315]:
grouped = df_binary_pred_full.groupby('query_id')['page_label_pred_score_pas', 'answerPage']
df_binary_pred_full2 = grouped.aggregate(lambda x: list(x))
df_binary_pred_full2

/anaconda/envs/pygaggle2/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,page_label_pred_score_pas,answerPage
query_id,,
0,"[[14, 0, 0, 0.32823217, fcev: expanded and imp...","[[22], [22], [22], [22], [22], [22], [22], [22..."
1,"[[50, 1, 1, 0.52675074, with its suppliers. we...","[[50], [50], [50], [50], [50], [50], [50], [50..."
2,"[[31, 1, 0, 0.21207151, editorial policy conte...","[[31], [31], [31], [31], [31], [31], [31], [31..."
3,"[[20, 1, 1, 0.7730109, to curb global warming ...","[[20], [20], [20], [20], [20], [20], [20], [20..."
4,"[[12, 1, 0, 0.19632585, editorial policy conte...","[[12], [12], [12], [12], [12], [12], [12], [12..."
...,...,...
8345,"[[4, 1, 1, 0.91164726, from the employee’s pay...","[[4], [4], [4], [4], [4], [4], [4], [4], [4], ..."
8346,"[[3, 1, 1, 0.8561231, to the end of the curren...","[[2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3..."
8347,"[[0, 1, 1, 0.917345, annual work time for ever...","[[0], [0], [0], [0], [0], [0], [0]]"


In [316]:
def sort(sub_li):  
    sub_li.sort(key = lambda x: x[3], reverse = True)
    return sub_li

df_binary_pred_full2['page_label_pred_score_pas'] = df_binary_pred_full2['page_label_pred_score_pas'].apply(lambda x: sort(x))

In [317]:
def pos_passages(top_passages, col_answerPage):
    pos_passages = [top_passages[i] for i in range(len(top_passages)) if str(passages_dict[top_passages[i][4]]) in col_answerPage[0]]
    return pos_passages

def neg_passages(top_passages, col_answerPage):
    neg_passages = [top_passages[i] for i in range(len(top_passages)) if str(passages_dict[top_passages[i][4]]) not in col_answerPage[0]]
    return neg_passages

df_binary_pred_full2['elecPosPassages'] = df_binary_pred_full2.apply(lambda x: pos_passages(x['page_label_pred_score_pas'], x['answerPage']), axis=1)
df_binary_pred_full2['elecNegPassages'] = df_binary_pred_full2.apply(lambda x: neg_passages(x['page_label_pred_score_pas'], x['answerPage']), axis=1)
df_binary_pred_full2

,page_label_pred_score_pas,answerPage,elecPosPassages,elecNegPassages
query_id,,,,
0,"[[27, 0, 1, 0.8353416, the assembly location d...","[[22], [22], [22], [22], [22], [22], [22], [22...",[],"[[27, 0, 1, 0.8353416, the assembly location d..."
1,"[[51, 0, 1, 0.9192894, following the measures ...","[[50], [50], [50], [50], [50], [50], [50], [50...","[[50, 1, 1, 0.85095733, in harmony with nature...","[[51, 0, 1, 0.9192894, following the measures ..."
2,"[[32, 0, 1, 0.9236114, fy 2015 2016 2017 2018 ...","[[31], [31], [31], [31], [31], [31], [31], [31...","[[31, 1, 0, 0.21207151, editorial policy conte...","[[32, 0, 1, 0.9236114, fy 2015 2016 2017 2018 ..."
3,"[[27, 0, 1, 0.9051206, the assembly location d...","[[20], [20], [20], [20], [20], [20], [20], [20...","[[20, 1, 1, 0.79130405, march 2019. and, by fu...","[[27, 0, 1, 0.9051206, the assembly location d..."
4,"[[22, 0, 1, 0.86297953, stage. in this system,...","[[12], [12], [12], [12], [12], [12], [12], [12...","[[12, 1, 1, 0.64752394, the introduction of an...","[[22, 0, 1, 0.86297953, stage. in this system,..."
...,...,...,...,...
8345,"[[4, 1, 1, 0.93089515, the first 3% contribute...","[[4], [4], [4], [4], [4], [4], [4], [4], [4], ...","[[4, 1, 1, 0.93089515, the first 3% contribute...","[[3, 0, 1, 0.9179397, for which they may be qu..."
8346,"[[3, 1, 1, 0.9193483, vacation time. immediate...","[[2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3...","[[3, 1, 1, 0.9193483, vacation time. immediate...","[[1, 0, 1, 0.9069631, revised 6.28.19 2 family..."
8347,"[[0, 1, 1, 0.9246664, 11 vocational profession...","[[0], [0], [0], [0], [0], [0], [0]]","[[0, 1, 1, 0.9246664, 11 vocational profession...","[[1, 0, 0, 0.0321198, promotes and sells its p..."


In [266]:
df_binary_pred_full2[df_binary_pred_full2.astype(str)['elecPosPassages'] == '[]']

,page_label_pred_score_pas,answerPage,elecPosPassages,elecNegPassages
query_id,,,,
0,"[[27, 0, 1, 0.8353416, the assembly location d...","[[22], [22], [22], [22], [22], [22], [22], [22...",[],"[[27, 0, 1, 0.8353416, the assembly location d..."
15,"[[25, 0, 1, 0.9233561, completed products comp...","[[22], [22], [22], [22], [22], [22], [22], [22...",[],"[[25, 0, 1, 0.9233561, completed products comp..."
84,"[[54, 0, 1, 0.5602695, and 1 june 2018 among 6...","[[53], [53], [53], [53], [53], [53], [53], [53...",[],"[[54, 0, 1, 0.5602695, and 1 june 2018 among 6..."
98,"[[48, 0, 1, 0.85073346, the city’s energy need...","[[46], [46], [46], [46], [46], [46], [46], [46...",[],"[[48, 0, 1, 0.85073346, the city’s energy need..."
101,"[[90, 0, 1, 0.9138939, 931,504 958,647 1,840,8...","[[89], [89], [89], [89], [89], [89], [89], [89...",[],"[[90, 0, 1, 0.9138939, 931,504 958,647 1,840,8..."
...,...,...,...,...
8298,"[[2, 0, 1, 0.8284284, for customers your indus...","[[], [], [], [], [], [], [], [], [], [], [], [...",[],"[[2, 0, 1, 0.8284284, for customers your indus..."
8309,"[[2, 0, 1, 0.862702, del monte fresh produce i...","[[], [], [], [], [], [], [], [], [], [], [], [...",[],"[[2, 0, 1, 0.862702, del monte fresh produce i..."
8312,"[[11, 0, 1, 0.89432317, (i) a description of t...","[[], [], [], [], [], [], [], [], [], [], [], [...",[],"[[11, 0, 1, 0.89432317, (i) a description of t..."


In [263]:
# Importing Electra Positive and Negative passages into Electra dataset
df_electra['elecPosPassages'] = df_binary_pred_full2['elecPosPassages']
df_electra['elecNegPassages'] = df_binary_pred_full2['elecNegPassages']

In [ ]:
# Creating a dataset with triples (query, positive passage, negative passage) for 2-way classification
import random 

rows_list = []
count = 0
for row in range(len(df_electra)):
    for i in range(min(3,len(df_electra['elecPosPassages'].iloc[row]))):
        if len(df_electra.elecNegPassages.iloc[row]) > 20:
            neg_samples_idx = random.sample(range(10,len(df_electra.elecNegPassages.iloc[row])), min(10, len(df_electra.elecNegPassages.iloc[row])))
        else:
            neg_samples_idx = random.sample(range(len(df_electra.elecNegPassages.iloc[row])), min(10, len(df_electra.elecNegPassages.iloc[row])))
        for j in neg_samples_idx:
            dict_example = {}
            dict_example.update({'query': df_electra['query'].iloc[row]}) 
            dict_example.update({'query_id': df_electra['query_id'].iloc[row]}) 
            dict_example.update({'positive': df_electra['elecPosPassages'].iloc[row][i][4]})
            dict_example.update({'negative': df_electra['elecNegPassages'].iloc[row][j][4]})
            dict_example.update({'answerPage': df_electra['answerPage'].iloc[row]}) 
            dict_example.update({'label': 0}) 
            dict_example.update({'example_id': count}) 
            count += 1
            rows_list.append(dict_example)

df_triple_examples_small = pd.DataFrame(rows_list, columns = ['query', 'query_id', 'positive', 'negative', 'answerPage', 'label', 'example_id'])
df_triple_examples_small['query_pos_neg'] = df_triple_examples_small.apply(lambda x: [[x['query'],x['positive']], [x['query'], x['negative']]], axis=1)

In [135]:
# Creating training and test sets based on query_ids and supplier_ids
df_triple_examples_small_query_train = df_triple_examples_small[df_triple_examples_small['query_id'].isin(train_query_ids)]
df_triple_examples_small_query_test = df_triple_examples_small[df_triple_examples_small['query_id'].isin(test_query_ids)]

df_triple_examples_small_supplier_train = df_triple_examples_small[df_triple_examples_small['query_id'].isin(train_supplierIds_queryIds)] 
df_triple_examples_small_supplier_test = df_triple_examples_small[df_triple_examples_small['query_id'].isin(test_supplierIds_queryIds)] 

df_triple_examples_small_query_train.to_csv('df_triple_examples_small_query_train.csv', index=False)
df_triple_examples_small_query_test.to_csv('df_triple_examples_small_query_test.csv', index=False)

df_triple_examples_small_supplier_train.to_csv('df_triple_examples_small_supplier_train.csv', index=False)
df_triple_examples_small_supplier_test.to_csv('df_triple_examples_small_supplier_test.csv', index=False)

In [136]:
df_triple_examples_small_supplier_train[df_triple_examples_small_supplier_train.astype(str)['positive'] == '[]']

,query,query_id,positive,negative,answerPage,label,example_id,query_pos_neg


### Electra 2-way classifier training

In [ ]:
epochs = 6
batch_size = 128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

elec_tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')


def preprocess(raw_text, labels=None):
    raw_text_bin = [i for j in raw_text for i in j]
    encoded_batch = elec_tokenizer.batch_encode_plus(raw_text_bin,
                                                truncation=True,
                                                add_special_tokens=True,
                                                padding='max_length',
                                                max_length = 256,
#                                                padding='longest',
                                                return_attention_mask=True,
                                                return_tensors = 'pt')
    labels_stacked=torch.stack([labels] * len(raw_text_bin))
    return encoded_batch['input_ids'], encoded_batch['attention_mask'], labels_stacked


def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))    
    return str(datetime.timedelta(seconds=elapsed_rounded))

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


df_triple_examples = pd.read_csv('df_triple_examples_small_supplier_train.csv')
df_triple_examples['query_pos_neg'] = df_triple_examples.apply(lambda x: [[x['query'],x['positive']], [x['query'], x['negative']]], axis=1)
text = df_triple_examples['query_pos_neg']
labels = torch.tensor(0)

df_triple_examples_test = pd.read_csv('df_triple_examples_small_supplier_test.csv')
df_triple_examples_test['query_pos_neg'] = df_triple_examples_test.apply(lambda x: [[x['query'],x['positive']], [x['query'], x['negative']]], axis=1)
test_inputs = df_triple_examples_test['query_pos_neg']


train_inputs, validation_inputs = train_test_split(text,
                                                            random_state=42, test_size=0.2)


train_input_ids, train_attention_mask, train_labels = preprocess(train_inputs, labels)

# Combine the training inputs into a TensorDataset
train_dataset = TensorDataset(
    train_input_ids,
    train_attention_mask,
    train_labels)

val_input_ids, val_attention_mask, validation_labels = preprocess(validation_inputs, labels)
# Combine the validation inputs into a TensorDataset
validation_dataset = TensorDataset(
    val_input_ids,
    val_attention_mask,
    validation_labels)

test_input_ids, test_attention_mask, test_labels = preprocess(test_inputs, labels)
# Combine the validation inputs into a TensorDataset
test_dataset = TensorDataset(
    test_input_ids,
    test_attention_mask,
    test_labels)

"""
torch.save(train_dataset, 'train_dataset_mc_small')
torch.save(validation_dataset, 'validation_dataset_mc_small')
torch.save(test_dataset, 'test_dataset_mc_small')


train_dataset = torch.load('train_dataset_mc_small')
validation_dataset = torch.load('validation_dataset_mc_small')
test_dataset = torch.load('test_dataset_mc_small')
"""

# Create the DataLoaders
train_dataloader = DataLoader(
            train_dataset,
            sampler = SequentialSampler(train_dataset),
            batch_size = batch_size)

validation_dataloader = DataLoader(
            validation_dataset,
            sampler = SequentialSampler(validation_dataset),
            batch_size = batch_size)

test_dataloader = DataLoader(
            test_dataset,
            sampler = SequentialSampler(test_dataset),
            batch_size = batch_size)

try:
    state_dict = torch.load("Electra_mc.pt")
    print("Loading trained model...")
    model = ElectraForSequenceClassification.from_pretrained('google/electra-small-discriminator',
    state_dict=state_dict)
    print("Trained model loaded!")
except Exception as e:
    print("Unable to load trained model.")
    print(e)
    model = ElectraForSequenceClassification.from_pretrained('google/electra-small-discriminator')

model.to(device) 

#Training parameters
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # Learning Rate - Default is 5e-5
                  eps = 1e-8 # Adam Epsilon  - Default is 1e-8.
                )

criterion = nn.CrossEntropyLoss()

# Training loop
training_stats = []
validation_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# Total number of training steps is [number of batches] x [number of epochs]
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

# This variable will evaluate the convergence on the training
consecutive_epochs_with_no_improve = 0

# Training & Validation
for epoch in range(0, epochs):
    print("")
    print(f'########## Epoch {epoch+1} / {epochs} ##########')
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode
    model.train()

    # For each batch of training data
    for step, batch in enumerate(train_dataloader):
    # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = time.time() - t0

            # Report progress
            print(f'  Batch {step}  of  {len(train_dataloader)}    Elapsed: {format_time(elapsed)}.')

        # Unpack this training batch from our dataloader. 
        input_id = batch[0].to(device)
        attention_mask = batch[1].to(device)
        label = batch[2].to(device)

        # Clear any previously calculated gradients before performing a backward pass
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch)
        output = model(input_id, 
                              token_type_ids=None, 
                              attention_mask=attention_mask, 
                              return_dict=False)

        scores = output[0][:,1]
        scores = scores.view(-1, 2)

        loss = criterion(scores, label[:scores.size(0)])

        # Accumulate the training loss over all of the batches
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients
        loss.backward()

        # Clip the norm of the gradients to 1.0
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches
    avg_train_loss = total_train_loss / len(train_dataloader)   

    if epoch > 0:
        if min([stat['Training Loss'] for stat in training_stats]) <= avg_train_loss:
            # i.e. If there is not improvement
            consecutive_epochs_with_no_improve += 1
        else:
            # If there is improvement
            consecutive_epochs_with_no_improve = 0
            print("Model saved!")
            torch.save(model.state_dict(), "Electra_mc_sup.pt")

    # Measure how long this epoch took
    training_time = time.time() - t0

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))

    # Record all statistics from this epoch.
    training_stats.append(
      {
          'epoch': epoch + 1,
          'Training Loss': avg_train_loss,
          'Training Time': training_time,
      }
    )

    print("epoch: {} \n".format(epoch + 1), file=outfile)
    print("Training Loss: {} \n".format(avg_train_loss), file=outfile)
    print("Training Time: {} \n".format(training_time), file=outfile)

    if consecutive_epochs_with_no_improve == 2:
        print("Stop training : The loss has not changed since 2 epochs!")
        break


    # Tracking variables for validation
    eval_loss, eval_accuracy = 0, 0
    batch_count = 0

    t1 = time.time()
    # Validation of the model
    model.eval()
    print('Evaluation...')

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        if batch_count % 40 == 0 and not batch_count == 0:
            # Calculate elapsed time in minutes.
            elapsed = time.time() - t1

            # Report progress
            print(f'  Batch {batch_count}  of  {len(validation_dataloader)}    Elapsed: {format_time(elapsed)}.')

        # Unpack this training batch from our dataloader. 
        input_id = batch[0].to(device)
        attention_mask = batch[1].to(device)
        label = batch[2].to(device)

        with torch.no_grad():
        # Forward pass, calculate logit predictions
            output = model(input_id, 
                                  token_type_ids=None, 
                                  attention_mask=attention_mask, 
                                  return_dict=False)

            scores = output[0][:,1]
            scores = scores.view(-1, 2)
            labels = label[:scores.size(0)]
            loss = criterion(scores, labels)

        # Accumulate the training loss over all of the batches
        eval_loss += loss.item()

        # Move logits and labels to CPU if GPU is used
        logits = scores.detach().cpu().numpy()
        label_ids = labels.to('cpu').numpy()
        predictions = np.argmax(logits, axis=1).flatten()
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)

        eval_accuracy += tmp_eval_accuracy
        if batch_count == 0:
            global_predictions = predictions
        else:
            global_predictions = np.append(global_predictions,predictions)
        batch_count += 1


    avg_validation_loss = eval_loss / len(validation_dataloader)   


    # Measure how long this epoch took
    validation_time = time.time() - t1

    print("")
    print("  Average validation loss: {0:.2f}".format(avg_validation_loss))
    print("  Validation took: {:}".format(validation_time))
    print("  Validation Accuracy: {}".format(eval_accuracy/batch_count))
    # Calculate the average loss over all of the batches

    # Record all statistics from this epoch.
    validation_stats.append(
      {
          'Training Loss': avg_validation_loss,
          'Training Time': validation_time,
      }
    )

    print("Validation Loss: {} \n".format(avg_validation_loss), file=outfile)
    print("Validation Time: {} \n".format(validation_time), file=outfile)


#Evaluating on dev set
print("Test mode: \n")

t2 = time.time()
model.eval()
batch_count = 0
test_loss, test_accuracy = 0, 0

for batch in test_dataloader:

    # Progress update every 40 batches.
    if batch_count % 40 == 0 and not batch_count == 0:
        # Calculate elapsed time in minutes.
        elapsed = time.time() - t2

        # Report progress
        print(f'  Batch {batch_count}  of  {len(test_dataloader)}    Elapsed: {format_time(elapsed)}.')

    input_id = batch[0].to(device)
    attention_mask = batch[1].to(device)
    label = batch[2].to(device)

    with torch.no_grad():
    # Forward pass, calculate logit predictions
        output = model(input_id, 
                              token_type_ids=None, 
                              attention_mask=attention_mask, 
                              return_dict=False)
        scores = output[0][:,1]
        scores = scores.view(-1, 2)
        labels = label[:scores.size(0)]
        loss = criterion(scores, labels)

    test_loss += loss.item()

    # Move logits and labels to CPU if GPU is used
    logits = scores.detach().cpu().numpy()
    label_ids = labels.to('cpu').numpy()
    predictions = np.argmax(logits, axis=1).flatten()

    tmp_test_accuracy = flat_accuracy(logits, label_ids)

    test_accuracy += tmp_test_accuracy
    if batch_count == 0:
        global_predictions = predictions
    else:
        global_predictions = np.append(global_predictions,predictions)
    batch_count += 1


print("Final Model saved!")

torch.save(model.state_dict(), "Electra_mc.pt")

avg_test_loss = test_loss / len(test_dataloader)   

# Measure how long this epoch took
test_time = time.time() - t2

global_preds = torch.from_numpy(global_predictions)

test_labels = test_labels[:int(len(test_labels)/2)]
report = metrics.classification_report(test_labels, global_preds, zero_division=0)

confusion_matrix = metrics.confusion_matrix(test_labels, global_preds)


print("  Average test loss: {0:.2f}".format(avg_test_loss))
print("  Test took: {:}".format(test_time))
print("  Test Accuracy: {}".format(test_accuracy/batch_count))
print("Report on test set: {} \n".format(report))
print("Confusion Matrix on test set: \n{} \n".format(confusion_matrix))

conf_plot = sns.heatmap(confusion_matrix/np.sum(confusion_matrix), annot=True)
fig = conf_plot.get_figure()

### Electra 2-way classifier evaluation

In [ ]:
# First step is to use previously trained Electra model to generate predictions on each example of df_triple_examples_small_query_test
# To do so, inference mode should be done in classical binary classification mode

#    with torch.no_grad():
    # Forward pass, calculate logit predictions
#        outputs =  model(input_id, token_type_ids=None, 
#                              attention_mask=attention_mask,
#                              labels=label
#                        )
#        loss, logits = outputs[:2]

#    test_loss += loss.item()
#    probs = F.softmax(logits, dim=1)

    # Move logits and labels to CPU if GPU is used
#    logits = logits.detach().cpu().numpy()
#    probs = probs.detach().cpu().numpy()
#    probs_list.extend([i for i in probs])

#    label_ids = label.to('cpu').numpy()
#    predictions = np.argmax(logits, axis=1).flatten()

In [ ]:
# The results of these predictions are stored in a dataframe for future evaluation

df_triple_test = df_triple_examples_small_query_test.copy()

df_bin_test = pd.DataFrame()
df_bin_test['query_pas'] = [i for j in df_triple_test['query_pos_neg'] for i in j]
df_bin_test['query'] = df_bin_test['query_pas'].apply(lambda x: x[0])
df_bin_test['passage'] = df_bin_test['query_pas'].apply(lambda x: x[1])

df_bin_test['probs'] = probs_list
df_bin_test['prediction'] = global_predictions

df_electra = pd.read_csv('df_electra.csv')
df_electra['passages'] = df_electra['passages'].apply(lambda x: ast.literal_eval(x))
passText_passPage_dict = {i[2]:i[0] for p in df_electra['passages'].tolist() for i in p}

answerPages_list = df_triple_test['answerPage'].apply(lambda x: [x,x])
df_bin_test['answerPage'] = [i for l in answerPages_list for i in l]

queryId_list = df_triple_test['query_id'].apply(lambda x: [x,x])
df_bin_test['query_id'] = [i for l in queryId_list for i in l]

df_bin_test['pass_page'] = df_bin_test['passage'].apply(lambda x: passText_passPage_dict[x])

df_bin_test['page_pred_score_pas'] = df_bin_test.apply(lambda x: [x['pass_page'], x['prediction'], x['probs'], x['passage']], axis=1)

df_bin_test.to_csv('df_bin_pred_test_mc_6e.csv', index=False)

In [2]:
# Reimporting the test dataset with results of Electra predictions to evaluate the model
import ast
df_triple_pred_test = pd.read_csv('df_bin_pred_test_mc_6e.csv')
#df_triple_pred_test = pd.read_csv('df_binary_test_mc_sup_6e.csv')
df_triple_pred_test['probs'] = df_triple_pred_test['probs'].apply(lambda x: [t for t in str(x).strip('][').split(' ') if t])
df_triple_pred_test['query_pass'] = df_triple_pred_test.apply(lambda x: str(x['query']) + str(x['passage']), axis=1)
df_triple_pred_test['answerPage'] = df_triple_pred_test['answerPage'].apply(lambda x: re.findall(r'\d+', x))
df_triple_pred_test['page_pred_score_pas'] = df_triple_pred_test.apply(lambda x: [x['pass_page'], x['prediction'], x['probs'][0], x['passage']], axis=1)
df_triple_pred_test

,query_pas,query,passage,probs,prediction,answerPage,query_id,pass_page,page_pred_score_pas,query_pass
0,['detailed information available on fuel consu...,detailed information available on fuel consump...,"stage. in this system, we conduct lca* which a...","[0.44416994, 0.55583006]",1,[22],0,22,"[22, 1, 0.44416994, stage. in this system, we ...",detailed information available on fuel consump...
1,['detailed information available on fuel consu...,detailed information available on fuel consump...,sixth toyota environmental action plan (fy2017...,"[0.86696273, 0.13303733]",0,[22],0,46,"[46, 0, 0.86696273, sixth toyota environmental...",detailed information available on fuel consump...
2,['detailed information available on fuel consu...,detailed information available on fuel consump...,"stage. in this system, we conduct lca* which a...","[0.96706367, 0.0329363]",0,[22],0,22,"[22, 0, 0.96706367, stage. in this system, we ...",detailed information available on fuel consump...
3,['detailed information available on fuel consu...,detailed information available on fuel consump...,top management's recognition of environmental ...,"[1.0124679e-04, 9.9989879e-01]",1,[22],0,9,"[9, 1, 1.0124679e-04, top management's recogni...",detailed information available on fuel consump...
4,['detailed information available on fuel consu...,detailed information available on fuel consump...,zero co2 emissions challenge the lca that toyo...,"[0.03213619, 0.9678638]",1,[22],0,22,"[22, 1, 0.03213619, zero co2 emissions challen...",detailed information available on fuel consump...
...,...,...,...,...,...,...,...,...,...,...
79735,['provision of additional leave beyond standar...,provision of additional leave beyond standard ...,handbook employee,"[2.6566561e-04, 9.9973434e-01]",1,"[2, 3]",8346,0,"[0, 1, 2.6566561e-04, handbook employee]",provision of additional leave beyond standard ...
79736,['monetary remuneration for employees beyond b...,monetary remuneration for employees beyond bas...,again committed to link reward more closely wi...,"[9.9964726e-01, 3.5278033e-04]",0,[0],8349,0,"[0, 0, 9.9964726e-01, again committed to link ...",monetary remuneration for employees beyond bas...
79737,['monetary remuneration for employees beyond b...,monetary remuneration for employees beyond bas...,be more agile and individually. to deliver our...,"[0.6558441, 0.34415588]",0,[0],8349,1,"[1, 0, 0.6558441, be more agile and individual...",monetary remuneration for employees beyond bas...
79738,['monetary remuneration for employees beyond b...,monetary remuneration for employees beyond bas...,2019 stvrs gkn powder metallurgy may 2019 2019...,"[9.9970394e-01, 2.9604996e-04]",0,[0],8349,0,"[0, 0, 9.9970394e-01, 2019 stvrs gkn powder me...",monetary remuneration for employees beyond bas...


In [3]:
df_triple_pred_test.drop_duplicates(subset ="query_pass",
                     keep = "first", inplace = True)
df_triple_pred_test

,query_pas,query,passage,probs,prediction,answerPage,query_id,pass_page,page_pred_score_pas,query_pass
0,['detailed information available on fuel consu...,detailed information available on fuel consump...,"stage. in this system, we conduct lca* which a...","[0.44416994, 0.55583006]",1,[22],0,22,"[22, 1, 0.44416994, stage. in this system, we ...",detailed information available on fuel consump...
1,['detailed information available on fuel consu...,detailed information available on fuel consump...,sixth toyota environmental action plan (fy2017...,"[0.86696273, 0.13303733]",0,[22],0,46,"[46, 0, 0.86696273, sixth toyota environmental...",detailed information available on fuel consump...
3,['detailed information available on fuel consu...,detailed information available on fuel consump...,top management's recognition of environmental ...,"[1.0124679e-04, 9.9989879e-01]",1,[22],0,9,"[9, 1, 1.0124679e-04, top management's recogni...",detailed information available on fuel consump...
4,['detailed information available on fuel consu...,detailed information available on fuel consump...,zero co2 emissions challenge the lca that toyo...,"[0.03213619, 0.9678638]",1,[22],0,22,"[22, 1, 0.03213619, zero co2 emissions challen...",detailed information available on fuel consump...
5,['detailed information available on fuel consu...,detailed information available on fuel consump...,"countries, major index from external rating ag...","[0.9471946, 0.05280543]",0,[22],0,9,"[9, 0, 0.9471946, countries, major index from ...",detailed information available on fuel consump...
...,...,...,...,...,...,...,...,...,...,...
79734,['provision of additional leave beyond standar...,provision of additional leave beyond standard ...,revised 6.28.19 3 *some states’ laws expand th...,"[0.00149207, 0.9985079]",1,"[2, 3]",8346,2,"[2, 1, 0.00149207, revised 6.28.19 3 *some sta...",provision of additional leave beyond standard ...
79735,['provision of additional leave beyond standar...,provision of additional leave beyond standard ...,handbook employee,"[2.6566561e-04, 9.9973434e-01]",1,"[2, 3]",8346,0,"[0, 1, 2.6566561e-04, handbook employee]",provision of additional leave beyond standard ...
79736,['monetary remuneration for employees beyond b...,monetary remuneration for employees beyond bas...,again committed to link reward more closely wi...,"[9.9964726e-01, 3.5278033e-04]",0,[0],8349,0,"[0, 0, 9.9964726e-01, again committed to link ...",monetary remuneration for employees beyond bas...
79737,['monetary remuneration for employees beyond b...,monetary remuneration for employees beyond bas...,be more agile and individually. to deliver our...,"[0.6558441, 0.34415588]",0,[0],8349,1,"[1, 0, 0.6558441, be more agile and individual...",monetary remuneration for employees beyond bas...


In [4]:
grouped = df_triple_pred_test.groupby('query_id')['page_pred_score_pas', 'answerPage']
df_triple_pred_final = grouped.aggregate(lambda x: list(x))
df_triple_pred_final

/anaconda/envs/pygaggle2/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,page_pred_score_pas,answerPage
query_id,,
0,"[[22, 1, 0.44416994, stage. in this system, we...","[[22], [22], [22], [22], [22], [22], [22]]"
1,"[[50, 1, 0.3680014, to protect human health an...","[[50], [50], [50], [50], [50], [50], [50], [50..."
2,"[[31, 1, 0.02081087, minimizing and optimizing...","[[31], [31], [31], [31], [31]]"
3,"[[20, 0, 9.9979347e-01, editorial policy conte...","[[20], [20], [20], [20], [20], [20], [20], [20]]"
4,"[[12, 1, 0.01804182, will contribute to climat...","[[12], [12], [12], [12], [12], [12], [12], [12..."
...,...,...
8343,"[[8, 1, 8.690386e-05, acting promptly to inves...","[[8], [8], [8], [8], [8], [8], [8], [8], [8]]"
8344,"[[8, 0, 0.52866644, 40 with the employee. “ext...","[[8], [8], [8], [8], [8], [8], [8], [8]]"
8345,"[[4, 0, 0.9984981, the first 3% contributed by...","[[4], [4], [4], [4], [4], [4], [4], [4]]"


In [5]:
def sort(sub_li):  
    sub_li.sort(key = lambda x: float(x[2]), reverse = True)
    return sub_li

df_triple_pred_final['page_pred_score_pas'] = df_triple_pred_final['page_pred_score_pas'].apply(lambda x: sort(x))
df_triple_pred_final['page_pred_score_pas'].iloc[0]

[[9,
  0,
  '0.9471946',
  'countries, major index from external rating agencies, and world leaders’ remarks on environmental issues at g7 summits. this broad examination provided us with an understanding of macroeconomic trends and important needs of societies, enabling us to organize candidate challenges. organize medium- to long-term issues based on the business environment toyota environmental challenge 2050 ―going beyond zero environmental impact and achieving a net positive impact we have formulated the toyota earth charter based on the guiding principles at toyota, considering environmental issues as a paramount importance, and have established its promotion structure to address such issues. in the course of perceiving'],
 [46,
  0,
  '0.86696273',
  'sixth toyota environmental action plan (fy2017–2021). toyota is identifying environmental risks and opportunities that can affect business operations and incorporating them into management plans to work for sustainable development 

In [6]:
# Scoring success on top k results
def predict_pages_electra(k, top_passages, answer_page):
    prediction = 0
    for j in range(k):
        try:
            if str(top_passages[j][0]) in answer_page[0]:
                prediction = 1 
        except:
            pass
    return prediction

# Running the evaluation on all lines except the ones with 1 page only
for k in [1,3,5,10]:
    results = df_triple_pred_final.apply(lambda x: predict_pages_electra(k, x['page_pred_score_pas'], x['answerPage']), axis=1)
    try:
        print("Success @ {}: {:f}".format(k, sum(results)/len(results)))
    except:
        print('Error, try lower k')

Success @ 1: 0.315832
Success @ 3: 0.736402
Success @ 5: 0.901606
Success @ 10: 0.952760


In [12]:
# Investigating the distribution of Success@3 depending on the type of report
import itertools

elec_query_ids_pred2 = list(df_triple_pred_final.index.values)
df_triple_pred_final2 = df_triple_pred_final.copy()
df_triple_pred_final2['typeName'] = df_electra[df_electra['query_id'].isin(elec_query_ids_pred2)]['typeName']
df_triple_pred_final2['numPages'] = df_electra[df_electra['query_id'].isin(elec_query_ids_pred2)]['numPages']
df_triple_pred_final2 = df_triple_pred_final2.reset_index()

grouped_type_elec = df_triple_pred_final2.groupby('typeName')['query_id', 'answerPage', 'numPages', 'page_pred_score_pas']
df_elec_full_type = grouped_type_elec.aggregate(lambda x: list(x))
df_elec_full_type = df_elec_full_type.reset_index()
df_elec_full_type['numPages'] = df_elec_full_type['numPages'].apply(lambda x: np.mean(x))
#df_bm25_full_type['query_id'] = df_bm25_full_type['query_id'].apply(lambda x: int(str(list(set(x))).strip('[]')))
df_elec_full_type['answerPage'] = df_elec_full_type['answerPage'].apply(lambda x: [i for i,_ in itertools.groupby(x)])
df_elec_full_type

elec_types = df_elec_full_type['typeName'].tolist()
print('Success @ 3 for type:')
for s in elec_types:
    index_s_elec = elec_types.index(s)
    query_idx_elec = df_elec_full_type.query_id.iloc[elec_types.index(s)]
    df_types_temp_elec = df_triple_pred_final2[df_triple_pred_final2['query_id'].isin(query_idx_elec)]
    results = df_types_temp_elec.apply(lambda x: predict_pages_electra(3, x['page_pred_score_pas'], x['answerPage']), axis=1)
    if len(results) != 0:
        print("  {} with average {:.0f} pages: {:f}".format(s, df_elec_full_type['numPages'].iloc[index_s_elec], sum(results)/len(results)))

Success @ 3 for type:
  Annual_Report with average 96 pages: 0.725622
  Audit_Report with average 33 pages: 0.838710
  CSR_Report with average 70 pages: 0.732711
  Certificate with average 15 pages: 1.000000
  Code_of_Ethics_Code_of_Conduct with average 21 pages: 0.738786
  Employee_Handbook with average 56 pages: 0.747204
  Letter_to_EcoVadis with average 82 pages: 0.600000
  Other with average 46 pages: 0.766304
  Policy with average 31 pages: 0.737024
  Procedure with average 19 pages: 0.769634
  Supplier_Code_of_Conduct with average 10 pages: 0.685714
  Webpage_Intranet_extract with average 6 pages: 0.745098


/anaconda/envs/pygaggle2/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  # This is added back by InteractiveShellApp.init_path()


In [13]:
# # Scoring MRR on top k results
def eval_MRR_electra(k, top_passages, answer_page):
    prediction = 0
    stop = False
    for j in range(k):
        try:
            if str(top_passages[j][0]) in answer_page[0] and stop == False:
                prediction = 1/(j+1)
                stop = True
        except:
            pass
    return prediction

# Running the evaluation 
for k in [1,3,5,10]:
    results = df_triple_pred_final.apply(lambda x: eval_MRR_electra(k, x['page_pred_score_pas'], x['answerPage']), axis=1)
    try:
        print("MRR @ {}: {:f}".format(k, sum(results)/len(results)))
    except:
        print('Error, try lower k')

MRR @ 1: 0.315832
MRR @ 3: 0.498110
MRR @ 5: 0.536483
MRR @ 10: 0.544268


In [14]:
# Investigating the distribution of Success@3 depending on the size of the documents

def page_range(x):
    if x < 31:
        pageRange = '2-30'
    elif x < 61:
        pageRange = '31-60'
    else:
        pageRange = '61-more'
    return pageRange

df_electra['pageRange'] = df_electra['numPages'].apply(lambda x: page_range(x))

import itertools

elec_query_ids_pred2b = list(df_triple_pred_final.index.values)
df_triple_pred_final2b = df_triple_pred_final.copy()
df_triple_pred_final2b['pageRange'] = df_electra[df_electra['query_id'].isin(elec_query_ids_pred2b)]['pageRange']
df_triple_pred_final2b = df_triple_pred_final2b.reset_index()

grouped_range_elec2 = df_triple_pred_final2b.groupby('pageRange')['query_id', 'answerPage', 'page_pred_score_pas']
df_elec_full_range2 = grouped_range_elec2.aggregate(lambda x: list(x))
df_elec_full_range2 = df_elec_full_range2.reset_index()
df_elec_full_range2['answerPage'] = df_elec_full_range2['answerPage'].apply(lambda x: [i for i,_ in itertools.groupby(x)])
df_elec_full_range2

elec_ranges2 = df_elec_full_range2['pageRange'].tolist()
print('Success @ 3 for type:')
for s in elec_ranges2:
    index_s_elec = elec_ranges2.index(s)
    query_idx_elec = df_elec_full_range2.query_id.iloc[elec_ranges2.index(s)]
    df_ranges_temp_elec2 = df_triple_pred_final2b[df_triple_pred_final2b['query_id'].isin(query_idx_elec)]
    results = df_ranges_temp_elec2.apply(lambda x: predict_pages_electra(3, x['page_pred_score_pas'], x['answerPage']), axis=1)
    if len(results) != 0:
        print("  {}: {:f}".format(s, sum(results)/len(results)))

Success @ 3 for type:
  2-30: 0.740722
  31-60: 0.722479
  61-more: 0.743025


/anaconda/envs/pygaggle2/lib/python3.6/site-packages/ipykernel_launcher.py:21: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


In [15]:
# Manual inspection
import random
df_temp = df_triple_pred_final.copy()
df_temp = df_temp.reset_index()
idx = random.randint(0, len(df_temp)-1)
query_id = df_temp['query_id'].iloc[idx]
print('Index: ', idx)
print('Query Id: ',query_id)
print('Query: ',df_electra.loc[df_electra['query_id'] == query_id, 'query'].iloc[0])
print('Query type: ', df_electra.loc[df_electra['query_id'] == query_id, 'query_type'].iloc[0])
print('\n')
print('Answer pages by Analysts: ', df_temp['answerPage'].iloc[idx][0])
print('Total number of pages of document: ', df_electra.loc[df_electra['query_id'] == query_id, 'numPages'].iloc[0])
print('Type of document: ', df_electra.loc[df_electra['query_id'] == query_id, 'typeName'].iloc[0])
print("\n")
print("Top passages:")
for i in range(min(10, len(df_temp['page_pred_score_pas']))):
    print(df_temp['page_pred_score_pas'].iloc[idx][i])
    print("\n")


Index:  2838
Query Id:  3163
Query:  company monitoring mechanism to keep track of the status of direct and indirect ghg emissions generated from company's operation and identify areas of improvement.
Query type:  option_help


Answer pages by Analysts:  ['22']
Total number of pages of document:  40.0
Type of document:  CSR_Report


Top passages:
[22, 0, '9.9976176e-01', '(383 thousand tco 2 e), and energy (23 thousand tco 2 e). this class of indirect emissions represents a new challenge for corporations like dystar. the company will increasingly focus on partnerships with all upstream and downstream businesses and suppliers to make quantifiable change in this area. the methodology employed to quantify scope 1, scope 2 and scope 3 emissions is in accordance with the greenhouse gas protocol corporate standard, developed by the world resources institute (wri) and world business council for sustainable development. scope 1 emissions are selected for reporting based on their presence in co

IndexError: list index out of range

In [16]:
def get_elec_score(qId, pText):
    df_temp = df_triple_pred_final.copy()
    df_temp = df_temp.reset_index()
    possible_passages = df_temp[df_temp['query_id'] == qId]['page_pred_score_pas'].iloc[0]
    temp_passText_score_dict = {i[3]:i[2] for i in possible_passages}
    return temp_passText_score_dict[pText]


def get_T5_score(qId, pText):
    possible_passages = df_T5_100[df_T5_100['query_id'] == qId]['T5Top100Passages'].iloc[0]
    temp_passText_score_dict = {i[3]:i[2] for i in possible_passages}
    return temp_passText_score_dict[pText]

In [20]:
pass_to_check = '(383 thousand tco 2 e), and energy (23 thousand tco 2 e). this class of indirect emissions represents a new challenge for corporations like dystar. the company will increasingly focus on partnerships with all upstream and downstream businesses and suppliers to make quantifiable change in this area. the methodology employed to quantify scope 1, scope 2 and scope 3 emissions is in accordance with the greenhouse gas protocol corporate standard, developed by the world resources institute (wri) and world business council for sustainable development. scope 1 emissions are selected for reporting based on their presence in company operations. hence, co'
qId = 3163

In [21]:
print(get_T5_score(qId, pass_to_check))
print(get_elec_score(qId, pass_to_check))

-0.20595459640026093
9.9976176e-01
